In [1]:
!pip install pynrrd
!pip install medpy

     |████████████████████████████████| 151 kB 5.4 MB/s 
     |████████████████████████████████| 48.4 MB 35 kB/s 
  Created wheel for medpy: filename=MedPy-0.4.0-cp37-cp37m-linux_x86_64.whl size=754465 sha256=326104bbecca1de297a0dc7ac3d23491b7708cfa2594e08161bfd175e8ce5c01
  Stored in directory: /root/.cache/pip/wheels/b0/57/3a/da1183f22a6afb42e11138daa6a759de233fd977a984333602
Successfully built medpy


In [2]:
import os
import re
import numpy as np
import pandas as pd
from shutil import copy
from tqdm.auto import tqdm

from nrrd import read, write
from medpy.io import load, save

In [3]:
!rm -rf /content/corrFiles
!rm -rf /content/correctedFiles

In [4]:
csv_data = pd.read_csv("/content/Heart_and_kidney_correction.csv", skiprows=lambda x: x % 2)

HeartKidney_path = "/content/MScSegmentationCourse_HeartKidney/"
corrFiles_path = "/content/corrFiles/"
correctedFiles_path = "/content/correctedFiles/"

for dir in [corrFiles_path, correctedFiles_path]:
  os.mkdir(dir)

### get corr files

In [5]:
files = os.listdir(HeartKidney_path)

for file in files:
  if "corr" in file and file[0] != '.':
    copy(HeartKidney_path + file, corrFiles_path + file)

### make corrections

In [6]:
files = os.listdir(corrFiles_path)

for file in tqdm(files):

  # correct filenames
  fish_name = file.split('_')[-1]
  sample_id = file.split('_')[-2]
  ext = fish_name[fish_name.index('.'):]
  m = re.findall(r"\d", fish_name)
  account_id = ''.join(m)
  new_fname = account_id + '-' + sample_id + ext

  # correct markup
  account_id, sample_id = int(account_id), int(sample_id)
  mask = (csv_data['u'] == account_id) & (csv_data['i'] == sample_id)
  file_data = csv_data[mask]

  # check for badly segmented files
  if "bad" in file_data["notes"].values[0]:
    print(f"skipped {file}")
    continue

  #img, hdr = load(corrFiles_path + file)   # medpy
  img, hdr = read(corrFiles_path + file)    # pynrrd
  # change labels 3 and 8 to background
  img[(img==3) | (img==8)] = 0

  # correct files 13-513 and 13-677 which have label inconsistency
  # labels for 7 and 10 are swapped
  if (account_id == 13) and (sample_id==513 or sample_id==677):
    tmp = img.copy()
    img[tmp==10] = 7
    img[tmp==7] = 10

  # medpy results in big file (>300 mb) size without compresion
  # around 2 mb with compression
  #save(img, correctedFiles_path + new_fname, hdr=hdr, use_compression=True)

  # pynrrd results in file size < 1 mb
  write(correctedFiles_path + new_fname, img, header=hdr)

  0%|          | 0/22 [00:00<?, ?it/s]

skipped heartandkidney_corr_513_dimorph13.nrrd
skipped heartandkidney_corr_511_dimorph16.nrrd
skipped heartandkidney_corr_508_dimorph04.nrrd
skipped heartandkidney_corr_511_dimorph12.nrrd
skipped heartandkidney_corr_675_dimorph12.nrrd


In [7]:
!zip -r /content/correctedFiles.zip /content/correctedFiles

  adding: content/correctedFiles/ (stored 0%)
  adding: content/correctedFiles/15-516.nrrd (deflated 85%)
  adding: content/correctedFiles/01-1095.nrrd (deflated 78%)
  adding: content/correctedFiles/01-803.nrrd (deflated 80%)
  adding: content/correctedFiles/04-673.nrrd (deflated 82%)
  adding: content/correctedFiles/15-803.nrrd (deflated 82%)
  adding: content/correctedFiles/16-1099.nrrd (deflated 79%)
  adding: content/correctedFiles/04-513.nrrd (deflated 81%)
  adding: content/correctedFiles/04-1258.nrrd (deflated 79%)
  adding: content/correctedFiles/15-677.nrrd (deflated 85%)
  adding: content/correctedFiles/12-1099.nrrd (deflated 78%)
  adding: content/correctedFiles/01-508.nrrd (deflated 82%)
  adding: content/correctedFiles/13-677.nrrd (deflated 83%)
  adding: content/correctedFiles/13-1258.nrrd (deflated 79%)
  adding: content/correctedFiles/04-675.nrrd (deflated 80%)
  adding: content/correctedFiles/12-673.nrrd (deflated 84%)
  adding: content/correctedFiles/06-516.nrrd (def